In [ ]:
pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.3 MB/s eta 0:00:00


In [ ]:
import argparse
import os
from time import time

import numpy as np
import torch
import torch.nn as nn
import torchvision

from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from torchvision import datasets
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/CoTuning-main/module'


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/CoTuning-main/module


In [ ]:
from backbone import ResNet50_F, ResNet50_C
from relationship_learning import relationship_learning

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/CoTuning-main/utils'

/content/drive/MyDrive/Colab Notebooks/CoTuning-main/utils


In [ ]:
from transforms import get_transforms
from tools import AccuracyMeter, TenCropsTest

In [ ]:
ResNet50_F
ResNet50_C
relationship_learning
get_transforms
AccuracyMeter
TenCropsTest

<function tools.TenCropsTest(loader, net)>

In [ ]:

def get_writer(log_dir):
    return SummaryWriter(log_dir)



In [ ]:


gpu=0
seed=2020
batch_size=48
total_iter=9050
eval_iter=1000
save_iter=9000
print_iter=100

# dataset
data_path="/content/drive/MyDrive/Colab Notebooks/CoTuning-main/CUB_200_2011"
class_num=200
num_workers=2

# optimizer
lr=1e-3
gamma=0.1
nesterov=True
momentum=0.9
weight_decay=5e-4

    # experiment
root='.'
name='StochNorm'
trade_off=2.3
relationship_path='relationship.npy'
save_dir="model"
visual_dir="visual"

In [ ]:

def str2list(v):
    return v.split(',')


def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")


def get_data_loader():
    data_transforms = get_transforms(resize_size=256, crop_size=224)

    # build dataset
    train_dataset = datasets.ImageFolder(
        os.path.join(data_path, 'images'),
        transform=data_transforms['train'])
    determin_train_dataset = datasets.ImageFolder(
        os.path.join(data_path, 'images'),
        transform=data_transforms['val'])
    val_dataset = datasets.ImageFolder(
        os.path.join(data_path, 'images'),
        transform=data_transforms['val'])
    test_datasets = {
        'test' + str(i):
            datasets.ImageFolder(
                os.path.join(data_path, 'images'),
                transform=data_transforms["test" + str(i)]
        )
        for i in range(10)
    }

    # build dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=True)
    determin_train_loader = DataLoader(determin_train_dataset, batch_size=batch_size, shuffle=False,
                                       num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=True)
    test_loaders = {
        'test' + str(i):
            DataLoader(
                test_datasets["test" + str(i)],
                batch_size=4, shuffle=False, num_workers=num_workers
        )
        for i in range(10)
    }

    return train_loader, determin_train_loader, val_loader, test_loaders


In [ ]:

def set_seeds(seed):
    # Set random seeds for reproducibility
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def main():
    torch.cuda.set_device(gpu)
    set_seeds(seed)

    train_loader, determin_train_loader, val_loader, test_loaders = get_data_loader()

    # Define the neural network model
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.f_net = ResNet50_F(pretrained=True)
            self.c_net_1 = ResNet50_C(pretrained=True)
            self.c_net_2 = nn.Linear(self.f_net.output_dim, class_num)
            self.c_net_2.weight.data.normal_(0, 0.01)
            self.c_net_2.bias.data.fill_(0.0)

        def forward(self, x):
            feature = self.f_net(x)
            out_1 = self.c_net_1(feature)
            out_2 = self.c_net_2(feature)

            return out_1, out_2

    net = Net().cuda()

    if os.path.exists(relationship_path):
        print('Loading pre-computed relationship from {}.'.format(relationship_path))
        relationship = np.load(relationship_path)
    else:
        print('Computing relationship')

        def get_feature(loader):
            train_labels_list = []
            imagenet_labels_list = []

            for train_inputs, train_labels in tqdm(loader):
                net.eval()
                train_labels_list.append(train_labels)

                train_inputs, train_labels = train_inputs.cuda(), train_labels.cuda()
                imagenet_labels, _ = net(train_inputs)
                imagenet_labels = imagenet_labels.detach().cpu().numpy()

                imagenet_labels_list.append(imagenet_labels)

            all_train_labels = np.concatenate(train_labels_list, 0)
            all_imagenet_labels = np.concatenate(imagenet_labels_list, 0)

            return all_imagenet_labels, all_train_labels

        train_imagenet_labels, train_train_labels = get_feature(determin_train_loader)
        val_imagenet_labels, val_train_labels = get_feature(val_loader)
        relationship = relationship_learning(train_imagenet_labels, train_train_labels,
                                             val_imagenet_labels, val_train_labels)

        np.save(relationship_path, relationship)

    train(train_loader, val_loader, test_loaders, net, relationship)


In [ ]:

def train(train_loader, val_loader, test_loaders, net, relationship):
    # Get the length of the train loader
    train_len = len(train_loader) - 1
    train_iter = iter(train_loader)

    # Different learning rates for different layers
    params_list = [
        {"params": filter(lambda p: p.requires_grad, net.f_net.parameters())},
        {"params": filter(lambda p: p.requires_grad, net.c_net_1.parameters())},
        {"params": filter(lambda p: p.requires_grad, net.c_net_2.parameters()), "lr": lr * 10}
    ]

    # Optimizer setup
    optimizer = torch.optim.SGD(params_list, lr=lr, weight_decay=weight_decay,
                                momentum=momentum, nesterov=nesterov)

    # Learning rate scheduling
    milestones = [6000]
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones, gamma=gamma)

    # Check visual path
    visual_path = os.path.join(visual_dir, name)
    if not os.path.exists(visual_path):
        os.makedirs(visual_path)
    writer = get_writer(visual_path)

    # Check model save path
    save_path = os.path.join(save_dir, name)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for iter_num in range(total_iter):
        net.train()

        if iter_num % train_len == 0:
            train_iter = iter(train_loader)

        # Data Stage
        data_start = time()

        # Get the next batch of training inputs and labels
        train_inputs, train_labels = next(train_iter)

        # Convert labels to one-hot encoding and move data to GPU
        imagenet_targets = torch.from_numpy(relationship[train_labels]).cuda().float()
        train_inputs, train_labels = train_inputs.cuda(), train_labels.cuda()

        data_duration = time() - data_start

        # Calc Stage
        calc_start = time()

        # Forward pass
        imagenet_outputs, train_outputs = net(train_inputs)

        # Calculate losses
        ce_loss = nn.CrossEntropyLoss()(train_outputs, train_labels)
        imagenet_loss = - imagenet_targets * nn.LogSoftmax(dim=-1)(imagenet_outputs)
        imagenet_loss = torch.mean(torch.sum(imagenet_loss, dim=-1))
        loss = ce_loss + trade_off * imagenet_loss

        # Log losses
        writer.add_scalar('loss/ce_loss', ce_loss, iter_num)
        writer.add_scalar('loss/imagenet_loss', imagenet_loss, iter_num)
        writer.add_scalar('loss/loss', loss, iter_num)

        # Backpropagation and optimization
        net.zero_grad()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        calc_duration = time() - calc_start

        if iter_num % eval_iter == 0:
            # Evaluation on validation dataset
            acc_meter = AccuracyMeter(topk=(1,))
            with torch.no_grad():
                net.eval()
                for val_inputs, val_labels in tqdm(val_loader):
                    val_inputs, val_labels = val_inputs.cuda(), val_labels.cuda()
                    _, val_outputs = net(val_inputs)
                    acc_meter.update(val_outputs, val_labels)
                writer.add_scalar('acc/val_acc', acc_meter.avg[1], iter_num)
                print("Iter: {}/{} Val_Acc: {:2f}".format(iter_num, total_iter, acc_meter.avg[1]))
            acc_meter.reset()

        if iter_num % save_iter == 0 and iter_num > 0:
            # Evaluation on test dataset and model saving
            test_acc = TenCropsTest(test_loaders, net)
            writer.add_scalar('acc/test_acc', test_acc, iter_num)
            print("Iter: {}/{} Test_Acc: {:2f}".format(iter_num, total_iter, test_acc))
            checkpoint = {
                'state_dict': net.state_dict(),
                'iter': iter_num,
                'acc': test_acc,
            }
            torch.save(checkpoint, os.path.join(save_path, '{}.pkl'.format(iter_num)))
            print("Model Saved.")

        if iter_num % print_iter == 0:
            # Print progress
            print("Iter: {}/{} Loss_main: {:2f}, d/c: {}/{}".format(iter_num, total_iter, loss,
                                                                     data_duration, calc_duration))




In [ ]:
if __name__ == '__main__':
    # print("PyTorch {}".format(torch.__version__))
    # print("TorchVision {}".format(torchvision.__version__))
    main()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 236MB/s]


Loading pre-computed relationship from relationship.npy.


100%|██████████| 246/246 [27:30<00:00,  6.71s/it]


Iter: 0/9050 Val_Acc: 0.567412
Iter: 0/9050 Loss_main: 21.235962, d/c: 3.4217073917388916/2.002387285232544
Iter: 100/9050 Loss_main: 10.397576, d/c: 0.4138517379760742/0.16207289695739746
Iter: 200/9050 Loss_main: 10.094559, d/c: 0.31432652473449707/0.1681227684020996
Iter: 300/9050 Loss_main: 8.873284, d/c: 0.32572054862976074/0.1576075553894043
Iter: 400/9050 Loss_main: 7.709411, d/c: 0.281203031539917/0.1657414436340332
Iter: 500/9050 Loss_main: 8.347646, d/c: 0.9492826461791992/0.1599268913269043
Iter: 600/9050 Loss_main: 7.565969, d/c: 0.7830579280853271/0.16496729850769043
Iter: 700/9050 Loss_main: 6.670202, d/c: 0.9346919059753418/0.16289162635803223
Iter: 800/9050 Loss_main: 6.389672, d/c: 0.6015768051147461/0.18138527870178223
Iter: 900/9050 Loss_main: 8.533734, d/c: 0.2918531894683838/0.18207359313964844


100%|██████████| 246/246 [01:49<00:00,  2.24it/s]


...

Iter: 8000/9050 Val_Acc: 99.432549
Iter: 8000/9050 Loss_main: 4.983940, d/c: 0.32604432106018066/0.16464447975158691
Iter: 8100/9050 Loss_main: 4.442134, d/c: 0.31666088104248047/0.1868443489074707
Iter: 8200/9050 Loss_main: 5.121646, d/c: 0.31320905685424805/0.15841269493103027
Iter: 8300/9050 Loss_main: 4.692663, d/c: 0.31890106201171875/0.16425609588623047
Iter: 8400/9050 Loss_main: 5.353915, d/c: 0.32420778274536133/0.16029572486877441
Iter: 8500/9050 Loss_main: 4.749154, d/c: 0.31821680068969727/0.1712348461151123
Iter: 8600/9050 Loss_main: 5.148423, d/c: 0.31516361236572266/0.16322660446166992
Iter: 8700/9050 Loss_main: 4.427621, d/c: 0.32207775115966797/0.15943598747253418
Iter: 8800/9050 Loss_main: 4.771709, d/c: 0.31677961349487305/0.1577284336090088
Iter: 8900/9050 Loss_main: 4.423179, d/c: 0.314708948135376/0.15876555442810059


100%|██████████| 246/246 [01:51<00:00,  2.21it/s]


Iter: 9000/9050 Val_Acc: 99.542664


100%|██████████| 2947/2947 [21:24<00:00,  2.30it/s]


Iter: 9000/9050 Test_Acc: 99.728539
Model Saved.
Iter: 9000/9050 Loss_main: 4.694311, d/c: 0.31838011741638184/0.15957331657409668


In [ ]:
# %cd r"/content/drive/MyDrive/Colab Notebooks/CoTuning-main/utils/model/StochNorm/9000.pkl"
# %cd r"/content/drive/MyDrive/Colab Notebooks/CoTuning-main/testing.jpg"

[Errno 2] No such file or directory: 'r/content/drive/MyDrive/Colab Notebooks/CoTuning-main/utils/model/StochNorm/9000.pkl'
/content/drive/MyDrive/Colab Notebooks/CoTuning-main/utils
[Errno 2] No such file or directory: 'r/content/drive/MyDrive/Colab Notebooks/CoTuning-main/testing.jpg'
/content/drive/MyDrive/Colab Notebooks/CoTuning-main/utils


In [ ]:
# import torch
# from torchvision import transforms
# from PIL import Image

# # Load the trained model checkpoint
# checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/CoTuning-main/utils/model/StochNorm/9000.pkl'
# checkpoint = torch.load(checkpoint_path)
# net = Net()
# net.load_state_dict(checkpoint['state_dict'])
# net.eval()

# # Define the image transformation
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# # Load and preprocess the image
# image_path = '/content/drive/MyDrive/Colab Notebooks/CoTuning-main/testing.jpg'
# image = Image.open(image_path).convert('RGB')
# image_tensor = transform(image).unsqueeze(0)

# # Perform the prediction
# with torch.no_grad():
#     outputs = net(image_tensor)

# # Process the prediction results
# _, predicted = torch.max(outputs[1], 1)
# class_index = predicted.item()

# # Load the class labels
# class_labels_path = '/path/to/your/class_labels.txt'
# with open(class_labels_path, 'r') as f:
#     class_labels = f.readlines()
# class_labels = [label.strip() for label in class_labels]

# # Get the predicted class label
# predicted_label = class_labels[class_index]

# # Print the predicted label
# print("Predicted class label:", predicted_label)


FileNotFoundError: ignored

In [ ]:
# # Example usage
# image_path = '/content/drive/MyDrive/Colab Notebooks/CoTuning-main/testing.jpg'  # Replace with the path to your image
# model_path = '/content/drive/MyDrive/Colab Notebooks/CoTuning-main/utils/model/StochNorm/9000.pkl'  # Replace with the path to your saved model


In [ ]:
# import argparse
# import os
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision
# from torch.utils.data import DataLoader
# from torchvision import datasets
# from tqdm import tqdm
# from PIL import Image

# # from module.backbone import ResNet50_F, ResNet50_C
# # from module.relationship_learning import relationship_learning
# # from utils.transforms import get_transforms
# # from utils.tools import TenCropsTest


# def get_data_loader():
#     data_transforms = get_transforms(resize_size=256, crop_size=224)

#     # build dataset
#     test_datasets = {
#         'test' + str(i):
#             datasets.ImageFolder(
#                 os.path.join(data_path, 'images'),
#                 transform=data_transforms["test" + str(i)]
#         )
#         for i in range(10)
#     }

#     # build dataloader
#     test_loaders = {
#         'test' + str(i):
#             DataLoader(
#                 test_datasets["test" + str(i)],
#                 batch_size=1, shuffle=False, num_workers=num_workers
#         )
#         for i in range(10)
#     }

#     return test_loaders


# def main():
#     # configs = get_configs()
#     # print(configs)

#     device = torch.device('cpu')

#     net = Net().to(device)
#     if os.path.exists(relationship_path):
#         print('load pre-computed relationship from {}.'.format(relationship_path))
#         relationship = np.load(relationship_path)
#     else:
#         print('computing relationship')
#         # ... relationship computation code ...

#     test_loaders = get_data_loader()
#     checkpoint_path = '/path/to/your/model.pkl'